## Work

### 1

In [7]:
# Not necessary, but just getting a sense of how big a grid would have to be to hold all the positions
sum([int(row[2:]) for row in split(s)])

11325

In [20]:
# Given an H location, T location, and a direction to move H, return the new H and T locations
def process_command(H: tuple, T: tuple, command: str):
    # For funsies
    assert len(H) == 2
    assert len(T) == 2
    assert len(command) == 1
    
    # Current locations and how we're moving H
    Hx, Hy = H
    Tx, Ty = T
    Dx, Dy = 0, 0
    
    # Requires Python 3.10
    match command:
        case 'U':
            Dy = 1
        case 'D':
            Dy = -1
        case 'R':
            Dx = 1
        case 'L':
            Dx = -1
    assert abs(Dx) + abs(Dy)
    
    # Move H
    Hp = (Hx + Dx, Hy + Dy)
    Hpx, Hpy = Hp
    
    # Move T: (just) 2 cases
    
    # No movement: they're still touching
    if max(abs(Hpx - Tx), abs(Hpy - Ty)) <= 1:
        Tp = T
    # Otherwise, T moves in the direction of H but with any 2s changed to 1s!
    else:
        Sx = Hpx - Tx
        Sy = Hpy - Ty
        
        if Sx:
            Sx = int(Sx / abs(Sx))
        if Sy:
            Sy = int(Sy / abs(Sy))
        
        Tp = (Tx + Sx, Ty + Sy)
    
    # Return
    return Hp, Tp

In [21]:
split(t)

['R 4', 'U 4', 'L 3', 'D 1', 'R 4', 'D 1', 'L 5', 'R 2']

In [22]:
# Process the instructions, mapping out the paths of H and T
H = (0,0)
T = (0,0)

Hs = [H]
Ts = [T]

for line in split(t):
    command = line[0]
    times = int(line[2:])
    for _ in range(times):
        H, T = process_command(H, T, command)
        Hs += [H]
        Ts += [T]

print(Hs)
print(Ts)

[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (3, 4), (2, 4), (1, 4), (1, 3), (2, 3), (3, 3), (4, 3), (5, 3), (5, 2), (4, 2), (3, 2), (2, 2), (1, 2), (0, 2), (1, 2), (2, 2)]
[(0, 0), (0, 0), (1, 0), (2, 0), (3, 0), (3, 0), (4, 1), (4, 2), (4, 3), (4, 3), (3, 4), (2, 4), (2, 4), (2, 4), (2, 4), (3, 3), (4, 3), (4, 3), (4, 3), (4, 3), (3, 2), (2, 2), (1, 2), (1, 2), (1, 2)]


In [25]:
len(set(Ts))

13

In [26]:
# Actual run
H = (0,0)
T = (0,0)

Hs = [H]
Ts = [T]

for line in split(s):
    command = line[0]
    times = int(line[2:])
    for _ in range(times):
        H, T = process_command(H, T, command)
        Hs += [H]
        Ts += [T]

len(set(Ts))

6026

### 2

My 2 -> 1 trick in process_command holds up! But I need to change it so we just move the location of things.

In [29]:
# Given the new location of H and the location of T, return the new T location
def update_knot(Hp: tuple, T: tuple):
    # For funsies
    assert len(Hp) == 2
    assert len(T) == 2
    
    # Locations
    Hpx, Hpy = Hp
    Tx, Ty = T
    
    # Move T: (just) 2 cases
    
    # No movement: they're still touching
    if max(abs(Hpx - Tx), abs(Hpy - Ty)) <= 1:
        Tp = T
    # Otherwise, T moves in the direction of H but with any 2s changed to 1s!
    else:
        Sx = Hpx - Tx
        Sy = Hpy - Ty
        
        if Sx:
            Sx = int(Sx / abs(Sx))
        if Sy:
            Sy = int(Sy / abs(Sy))
        
        Tp = (Tx + Sx, Ty + Sy)
    
    # Return
    return Tp

# And now for running the above on the whole chain
def process_command(knots: list, command: str):
    # For funsies
    assert len(knots) == 10
    assert len(command) == 1
    
    # For holding the future state
    knotsp = [None] * len(knots)
    
    # Grab the head
    Hx, Hy = knots[0]
    
    # Requires Python 3.10
    Dx, Dy = 0, 0
    match command:
        case 'U':
            Dy = 1
        case 'D':
            Dy = -1
        case 'R':
            Dx = 1
        case 'L':
            Dx = -1
    assert abs(Dx) + abs(Dy)
    
    # Update the head
    knotsp[0] = (Hx + Dx, Hy + Dy)
    
    # Iterate through the rest of the chain and update
    for i in range(1, 10):
        knotsp[i] = update_knot(knotsp[i-1], knots[i])
    
    # Return the state
    return knotsp

In [30]:
knots = [(0,0)] * 10
knots_history = [knots]

for line in split(tp):
    command = line[0]
    times = int(line[2:])
    for _ in range(times):
        knots = process_command(knots, command)
        knots_history += [knots]

In [36]:
len(set([knots[-1] for knots in knots_history]))

36

In [37]:
# Actual run

knots = [(0,0)] * 10
knots_history = [knots]

for line in split(s):
    command = line[0]
    times = int(line[2:])
    for _ in range(times):
        knots = process_command(knots, command)
        knots_history += [knots]
        
len(set([knots[-1] for knots in knots_history]))

2273

## Utilities

In [10]:
# Remove initial/final \n characters
def clean(s):
    return s[1:-1]

# Split at \n characters
# If there are \n\n characters, split into blocks too
def split(s):
    out = [block.split('\n') for block in clean(s).split('\n\n')]
    if len(out) == 1:
        return out[0]
    else:
        return out

# Apply a function(s) to a list or "block" data (2-level list)
def apply_func(data, func, nested=False):
    if not isinstance(func, list):
        func = [func]
        
    def _func(x):
        for f in func:
            x = f(x)
        return x
        
    if nested:
        return [[_func(x) for x in block] for block in data]
    else:
        return [_func(x) for x in data]

# Split, parsing everything as ints
def split_int(s):
    return apply_func(split(s), int)

# Split, parsing everything as float
def split_float(s):
    return apply_func(split(s), float)

## Input

In [11]:
t = """
R 4
U 4
L 3
D 1
R 4
D 1
L 5
R 2
"""

In [27]:
tp = """
R 5
U 8
L 8
D 3
R 17
D 10
L 25
U 20
"""

In [12]:
s = """
R 2
D 2
R 2
U 1
R 2
L 2
D 1
L 1
D 2
R 2
U 2
D 1
R 2
D 2
U 1
R 1
D 1
L 1
U 1
R 1
L 1
R 2
U 2
D 2
R 2
L 2
U 1
L 2
U 2
L 2
R 2
U 2
R 1
U 2
L 2
D 1
R 1
U 2
D 2
R 2
L 2
U 1
D 1
L 2
U 2
L 2
U 1
D 2
R 1
D 1
R 2
U 1
L 2
D 1
U 2
R 2
L 1
D 2
U 1
D 1
L 1
U 1
R 1
D 1
R 1
U 1
D 2
L 1
D 1
R 2
D 1
U 2
D 2
U 2
L 1
U 2
R 2
L 2
D 2
R 1
L 2
U 2
L 1
U 1
L 1
R 2
L 2
R 1
D 1
U 1
L 2
R 1
L 1
U 1
D 1
U 1
D 1
R 1
L 2
R 2
L 1
U 1
D 2
U 2
L 2
R 2
U 1
R 2
L 2
R 2
L 1
U 3
D 1
L 1
D 3
U 1
D 2
L 1
R 3
D 2
U 3
R 1
D 3
R 2
D 1
U 3
L 1
R 2
L 3
D 3
U 2
L 2
R 3
U 3
D 3
L 3
D 2
L 3
R 1
L 1
D 1
U 3
D 3
U 2
L 3
R 3
U 1
D 2
R 2
L 2
D 3
R 3
U 1
R 1
L 3
U 2
L 2
U 1
R 1
D 1
R 3
U 2
D 1
L 1
U 1
R 2
L 1
R 3
L 1
U 3
L 2
D 2
L 2
D 3
L 2
R 1
U 2
L 3
D 1
L 3
D 1
U 2
L 3
R 2
U 1
L 3
D 3
R 3
U 2
R 2
D 3
R 3
U 2
R 2
L 2
U 3
R 1
L 2
D 3
U 1
L 1
U 1
L 2
R 3
L 3
D 3
R 3
U 3
L 1
R 2
D 3
U 1
R 2
U 1
D 1
L 1
D 2
L 2
D 3
U 2
D 1
U 3
D 3
U 3
R 3
L 3
U 4
L 3
D 1
R 4
U 3
D 2
R 1
D 4
L 3
U 1
L 1
R 2
D 1
R 3
D 3
L 3
D 4
L 3
R 4
U 3
R 3
D 1
R 3
D 3
R 3
D 4
U 4
R 2
D 3
R 3
D 2
U 1
R 2
D 2
U 2
D 3
L 2
R 4
D 4
U 4
R 3
U 4
R 1
U 2
R 4
D 2
L 4
R 4
U 3
L 2
U 2
L 3
R 3
L 1
U 4
L 3
U 3
L 3
U 1
R 2
U 3
R 2
L 3
U 1
D 3
R 4
D 1
R 2
L 1
D 1
U 2
D 3
R 4
U 1
L 3
R 4
U 4
L 4
R 2
D 1
U 4
R 1
D 3
L 3
D 1
R 3
D 2
U 2
L 1
U 2
D 3
L 1
U 3
D 3
L 3
U 2
L 3
U 2
R 1
D 1
R 1
L 1
D 2
R 3
D 2
L 2
R 3
D 5
R 5
U 4
D 5
U 5
D 4
R 4
D 2
R 5
L 4
R 2
L 2
R 5
U 5
R 1
D 2
R 1
L 3
D 5
U 3
D 4
R 3
U 4
D 1
U 4
L 5
R 1
U 4
L 5
U 3
D 4
L 2
D 4
U 4
L 2
D 1
R 3
D 1
R 2
D 4
R 5
L 1
U 4
L 4
U 3
L 1
U 1
R 4
U 3
L 1
R 3
D 5
U 2
R 4
D 3
L 2
D 4
R 5
D 2
U 3
R 4
D 4
L 2
U 4
R 5
L 1
U 5
D 1
R 3
L 4
D 1
U 3
D 5
R 2
D 2
L 5
U 2
D 3
L 5
D 5
L 1
R 5
D 2
U 3
R 2
L 2
U 1
L 5
R 1
L 4
D 5
R 5
D 2
U 5
L 5
D 1
L 1
D 3
R 1
D 2
L 2
R 4
U 5
L 2
R 1
U 3
D 1
U 1
D 5
R 2
D 2
R 3
D 6
U 1
R 6
D 4
U 1
R 5
L 6
R 2
L 6
U 6
R 5
D 2
U 1
D 2
L 2
D 5
R 3
U 4
D 2
L 1
U 1
D 3
U 1
D 1
R 5
U 1
R 1
D 3
U 5
L 6
R 3
L 6
U 4
D 2
R 1
L 6
R 2
D 3
U 2
R 2
L 1
D 1
U 1
R 4
U 1
R 2
U 2
L 2
U 2
D 5
L 4
U 6
L 3
D 6
U 5
L 6
U 3
R 5
U 2
R 4
L 6
D 1
R 5
D 4
L 2
R 2
U 1
D 2
L 2
U 4
R 1
D 2
U 2
R 6
L 2
D 5
R 5
L 3
D 1
U 1
L 2
D 4
R 6
L 5
R 3
U 1
R 3
L 1
R 3
U 1
L 1
U 4
D 4
U 3
R 3
U 1
R 2
D 1
L 6
U 2
R 2
D 1
R 5
L 2
D 5
L 3
R 6
D 2
L 2
D 3
L 5
D 7
L 7
D 4
U 5
R 7
L 7
D 3
U 5
R 6
U 3
L 6
R 5
L 4
D 4
R 4
L 6
D 6
U 3
D 3
R 7
U 1
L 5
U 3
L 4
U 3
L 5
D 1
R 7
U 3
L 6
D 1
U 3
L 2
U 6
L 6
D 1
U 1
D 2
L 5
U 4
R 5
D 7
U 3
D 4
L 1
D 3
R 7
D 2
U 4
L 1
D 3
R 3
D 6
R 4
U 3
D 4
R 6
L 7
U 5
L 7
R 3
L 7
D 5
R 7
D 4
U 3
R 6
U 7
D 2
R 2
L 2
D 1
R 3
U 3
D 5
U 3
L 3
D 6
L 6
R 1
L 5
D 6
U 4
R 2
L 2
D 1
U 7
L 5
D 3
U 1
L 1
R 1
L 6
R 1
D 6
U 4
L 7
U 6
R 6
L 2
D 5
R 1
D 6
L 1
R 3
D 1
U 5
L 1
D 3
U 7
D 5
U 7
D 7
U 4
R 3
U 4
L 8
U 1
R 1
D 2
U 5
D 2
L 5
D 6
U 4
R 5
L 5
U 7
L 3
D 1
L 2
R 8
L 1
U 8
L 2
D 3
R 6
L 8
D 2
L 1
U 4
D 1
R 2
D 7
U 5
D 4
U 4
L 2
D 6
U 1
D 5
U 2
D 2
L 8
D 6
L 2
U 1
R 7
D 1
U 5
L 6
D 3
L 2
D 8
R 6
L 5
U 2
L 6
U 4
D 6
L 7
U 8
L 5
D 8
R 8
D 6
L 3
U 6
L 1
D 8
R 7
L 7
R 2
D 6
R 1
U 5
L 5
R 5
L 3
U 8
L 1
U 2
D 5
R 8
U 5
L 5
D 2
R 5
U 1
D 8
R 3
U 1
R 6
D 6
R 5
D 6
L 8
U 8
L 8
U 8
L 8
R 8
U 8
R 5
L 1
D 3
U 1
D 6
U 1
L 8
U 3
R 5
L 2
R 1
D 2
L 2
U 4
D 1
R 1
D 2
R 2
U 4
R 3
U 8
D 9
L 9
D 5
L 4
D 7
L 9
U 4
L 9
D 7
U 4
L 3
U 2
R 5
U 3
L 1
R 5
L 8
U 4
R 6
L 8
U 1
R 7
D 6
R 6
U 7
L 9
R 7
U 8
R 5
U 6
L 4
D 4
L 3
R 8
U 6
L 7
R 4
D 7
R 3
L 3
D 2
L 2
U 5
R 8
L 7
R 9
D 8
U 1
L 1
D 4
U 6
R 4
L 2
R 9
L 9
R 2
D 3
L 4
U 6
R 3
D 5
L 4
R 8
U 3
R 5
L 3
D 7
R 1
D 2
U 5
L 7
D 6
R 1
D 9
L 9
R 8
U 8
L 8
D 7
U 3
L 2
U 4
D 8
U 6
R 3
U 5
R 7
D 1
R 8
L 9
U 7
D 9
L 2
R 7
U 1
R 6
D 3
L 5
U 10
L 8
R 3
L 4
D 9
U 4
L 5
U 5
D 10
R 8
U 6
L 3
U 4
D 6
L 8
D 2
R 8
L 9
U 3
R 5
D 1
R 9
D 6
L 5
R 10
L 8
R 9
L 6
U 8
L 8
U 5
R 3
U 6
D 3
L 10
D 9
U 5
L 3
U 6
R 2
D 7
R 9
L 5
D 3
U 3
R 2
D 10
U 3
R 2
U 6
L 10
R 3
U 3
R 4
L 8
R 1
L 6
U 9
L 7
D 7
R 6
U 3
R 1
L 6
D 5
L 3
U 2
L 5
R 9
D 5
R 4
L 2
U 5
D 7
L 10
D 9
U 7
D 9
R 5
D 3
L 1
R 10
L 4
U 6
R 7
L 2
R 6
L 3
R 3
D 2
L 1
R 2
L 7
D 8
R 5
L 7
D 8
U 5
L 8
U 4
R 8
U 1
D 1
U 4
D 6
R 8
U 10
D 5
U 5
L 1
D 2
R 10
U 10
R 2
L 9
R 2
D 3
L 6
R 10
D 10
R 9
D 7
U 3
R 4
D 9
R 4
D 9
U 9
L 8
D 7
R 7
U 7
R 7
D 9
L 7
U 5
L 10
R 8
L 11
D 9
L 5
R 11
L 6
R 1
U 2
D 9
L 3
R 6
L 6
R 2
L 6
D 6
R 2
D 5
L 8
R 9
U 6
D 6
L 1
R 3
D 2
R 5
D 6
L 1
D 10
U 2
R 4
U 6
R 11
D 1
R 2
L 4
D 9
L 4
U 8
D 10
U 4
R 10
D 1
U 11
R 2
L 11
D 7
U 1
L 7
U 10
D 10
L 11
R 1
U 8
R 1
L 11
R 3
D 10
R 10
L 9
U 7
R 9
L 6
R 7
U 5
R 6
U 4
L 11
U 7
R 7
L 4
U 4
R 2
D 3
R 2
L 9
D 4
L 11
R 6
D 2
L 9
R 7
D 6
U 5
L 8
D 9
R 11
U 11
R 6
D 11
L 9
R 4
L 6
U 10
R 11
U 5
L 6
R 12
D 3
R 4
U 12
R 6
D 8
R 1
D 2
L 11
R 6
D 5
U 10
D 5
R 8
L 6
D 5
U 5
R 8
L 1
D 10
R 5
U 6
L 8
R 4
L 10
D 6
R 3
D 9
L 4
R 9
U 5
R 2
U 11
R 9
U 8
R 7
L 2
D 7
U 7
D 8
U 12
R 6
D 12
R 5
D 10
U 2
D 8
U 6
R 3
U 5
R 5
L 11
D 10
R 3
U 11
D 8
R 7
L 3
U 9
R 4
L 2
D 4
R 8
D 9
R 2
D 3
U 6
L 5
D 4
L 2
R 9
U 5
D 1
U 9
R 9
U 11
L 12
R 10
D 5
U 1
R 9
U 4
D 8
R 1
D 5
R 7
U 11
L 6
U 1
R 12
D 1
R 7
U 12
D 9
R 12
L 1
U 3
D 6
R 11
L 6
U 8
D 11
R 7
U 5
D 4
U 1
L 11
D 1
U 5
R 13
L 9
U 4
D 4
R 10
U 4
D 7
L 4
D 10
R 6
D 5
R 9
L 10
D 2
U 11
D 8
U 1
R 8
U 5
R 8
D 10
L 10
D 7
R 5
U 6
D 7
U 9
R 8
D 4
R 2
D 1
L 1
U 2
R 10
D 2
U 13
L 9
U 13
L 12
U 12
D 6
L 1
R 4
L 7
R 4
L 2
U 8
R 7
D 10
U 2
L 8
D 5
R 9
D 9
R 2
L 3
U 2
D 11
U 5
D 12
R 3
U 10
D 2
R 1
D 13
U 7
L 7
R 1
U 1
R 1
U 6
D 12
R 7
L 7
R 9
U 10
R 13
D 3
R 12
U 1
L 6
R 9
U 10
L 1
U 13
R 2
D 8
U 12
D 10
L 7
U 6
D 9
L 11
D 8
U 4
R 1
D 11
L 8
U 1
D 4
R 7
U 7
R 5
L 1
R 14
L 8
D 10
R 13
U 13
R 9
L 13
U 2
D 7
R 11
D 2
L 9
D 11
L 2
U 12
D 11
R 12
D 3
R 6
L 11
R 7
D 3
U 6
D 9
L 10
R 13
D 7
U 14
L 10
R 11
D 12
R 12
U 13
R 4
L 14
U 5
L 4
U 10
L 3
U 13
R 9
D 7
U 8
R 5
L 14
U 5
R 6
D 13
L 13
U 9
R 13
D 1
L 8
R 9
U 8
L 7
U 4
D 4
R 4
L 5
U 11
R 14
L 2
U 1
D 14
R 13
D 3
L 13
D 2
U 9
D 14
U 1
R 1
D 12
L 6
R 4
U 4
L 11
U 14
D 13
R 12
L 7
U 13
R 4
U 14
L 14
R 11
L 8
D 3
U 4
R 1
U 5
D 12
L 5
U 14
D 6
R 6
U 4
L 1
D 11
U 9
R 5
U 13
D 7
L 5
D 3
L 8
U 7
D 4
L 13
R 1
D 11
L 7
U 6
D 9
R 10
D 6
R 9
L 2
D 6
U 12
L 10
D 3
U 6
L 4
R 4
U 12
D 13
R 9
L 1
R 5
U 14
R 14
D 7
U 10
R 3
L 4
U 6
L 13
U 14
R 10
L 7
D 12
U 5
L 11
D 7
U 9
D 4
L 1
R 11
D 9
L 5
D 15
R 4
U 12
L 1
U 8
L 15
D 15
L 3
R 11
L 9
R 1
U 12
D 9
R 7
D 6
U 2
L 12
U 13
R 11
L 15
D 4
L 8
U 6
L 7
R 2
U 7
D 2
U 13
D 10
L 13
U 1
D 6
L 11
R 14
L 5
U 2
R 11
D 10
L 4
R 14
U 13
D 9
U 2
D 4
R 14
L 4
R 4
L 4
U 6
D 11
L 10
U 9
L 3
U 12
L 8
U 9
D 4
U 4
L 10
U 1
D 8
U 15
D 3
U 12
R 3
D 8
U 1
R 10
L 1
U 5
D 1
L 1
R 3
U 1
R 8
U 10
R 1
U 11
R 15
D 16
U 13
D 7
U 7
D 14
R 9
L 13
R 12
U 2
R 11
L 11
R 3
D 11
U 11
R 2
D 3
U 6
L 14
R 6
L 6
U 3
R 3
D 1
U 5
L 3
R 4
D 9
U 15
R 2
U 5
D 5
R 14
U 6
R 1
D 4
U 2
D 3
R 8
L 4
D 2
U 12
D 2
U 7
L 14
U 1
R 10
L 4
D 12
L 9
D 16
U 2
R 7
D 16
U 9
D 13
U 7
L 14
D 5
R 12
U 13
R 13
U 2
L 11
D 9
U 2
R 8
U 5
R 7
U 9
L 11
D 16
U 10
R 5
D 7
L 6
D 8
U 4
R 3
D 5
L 5
U 11
R 4
D 12
R 12
L 10
R 15
U 9
D 12
L 11
U 6
D 11
U 12
L 6
R 3
U 12
D 5
L 1
R 12
U 14
L 1
D 2
L 4
D 7
L 12
U 10
L 12
U 4
D 14
R 13
U 9
L 2
U 6
L 1
U 9
D 16
L 2
R 16
D 5
L 7
U 13
L 15
R 1
D 8
R 15
D 5
R 12
L 5
R 11
U 17
D 13
U 13
L 4
D 8
L 1
D 7
R 11
U 7
R 11
L 9
D 3
L 2
U 4
D 16
L 10
U 11
R 5
U 4
R 6
L 11
R 17
U 13
R 12
D 11
L 17
U 15
D 4
R 9
U 9
L 10
D 5
R 14
U 10
R 5
U 15
L 12
D 14
R 14
L 11
R 5
U 7
D 1
R 4
D 14
U 3
R 6
D 5
R 1
D 12
L 14
D 6
L 15
U 16
D 10
R 12
D 9
U 1
D 1
L 4
D 6
U 16
R 11
L 11
R 3
L 2
U 12
R 9
D 16
R 12
L 14
U 6
L 7
D 5
U 4
R 13
L 8
D 16
U 5
D 5
U 6
R 14
L 6
D 7
U 3
L 15
R 14
L 17
R 17
U 16
L 4
R 3
U 1
D 15
L 12
D 11
L 18
D 18
U 10
D 4
U 1
R 11
U 18
D 12
R 11
U 13
D 14
R 17
D 18
U 13
R 8
U 4
R 9
D 18
L 6
R 7
L 14
U 10
L 10
U 16
D 15
L 15
D 3
L 2
D 7
R 3
D 7
U 11
L 6
R 8
L 3
U 3
R 8
D 8
U 10
L 15
D 1
U 16
R 1
L 13
R 7
U 10
L 12
R 8
D 3
L 14
D 12
R 12
D 3
L 2
D 15
R 3
L 14
D 4
R 5
L 13
D 9
U 2
L 5
R 13
D 4
L 14
R 14
D 16
R 1
U 11
R 6
L 7
U 16
L 15
D 6
U 7
R 16
D 2
L 15
D 18
U 3
D 1
R 7
D 1
L 4
D 16
U 12
L 12
R 7
U 5
L 5
U 13
R 13
U 7
L 9
R 9
U 8
D 6
L 3
U 4
R 12
L 12
D 16
U 12
L 4
R 11
U 9
D 13
R 15
L 3
D 17
R 6
D 13
R 6
D 5
L 19
U 10
D 3
L 2
U 13
R 2
D 3
L 3
U 3
R 1
U 12
L 2
D 19
L 17
R 10
D 11
U 8
R 10
L 15
R 4
D 9
U 4
D 11
U 4
D 3
L 5
D 15
U 1
L 19
D 16
R 11
U 12
D 16
L 4
R 13
U 2
L 19
U 9
R 5
D 7
R 19
L 9
U 2
D 2
R 3
U 10
L 13
D 16
L 2
D 5
L 8
R 9
L 18
R 9
U 13
D 7
R 14
U 8
R 18
L 18
U 8
L 13
R 9
U 5
L 16
U 4
D 4
R 10
U 13
R 2
L 6
R 18
D 3
L 11
R 12
L 2
R 12
D 7
U 12
R 13
U 14
D 1
R 8
L 13
U 5
L 9
U 19
D 14
L 4
U 8
D 8
U 1
D 16
U 4
D 8
U 20
"""